## RestNet50

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

## Transforms

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),     # ResNet expects 224x224
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225]), # ImageNet normalization
])

## Dataset Split

In [3]:
dataset = datasets.ImageFolder("face_dataset/Original Images", transform=transform)

train_size = int(0.7 * len(dataset))
val_size   = int(0.15 * len(dataset))
test_size  = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)

print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")
print("Classes:", dataset.classes)

Train: 1793, Val: 384, Test: 385
Classes: ['Akshay Kumar', 'Alexandra Daddario', 'Alia Bhatt', 'Amitabh Bachchan', 'Andy Samberg', 'Anushka Sharma', 'Billie Eilish', 'Brad Pitt', 'Camila Cabello', 'Charlize Theron', 'Claire Holt', 'Courtney Cox', 'Dwayne Johnson', 'Elizabeth Olsen', 'Ellen Degeneres', 'Henry Cavill', 'Hrithik Roshan', 'Hugh Jackman', 'Jessica Alba', 'Kashyap', 'Lisa Kudrow', 'Margot Robbie', 'Marmik', 'Natalie Portman', 'Priyanka Chopra', 'Robert Downey Jr', 'Roger Federer', 'Tom Cruise', 'Vijay Deverakonda', 'Virat Kohli', 'Zac Efron']


## Load Pretrained ResNet50

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(pretrained=True)

# Freeze earlier layers (optional, for speed)
for param in resnet.parameters():
    param.requires_grad = False

# Replace final FC layer
num_classes = len(dataset.classes)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

resnet = resnet.to(device)

/home/winswe/miniconda3/envs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/winswe/miniconda3/envs/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/winswe/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:21<00:00, 4.77MB/s]


## Training Setup

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)  # only train last layer

EPOCHS = 15
best_val_acc = 0.0

## Training Loop

In [6]:
for epoch in range(EPOCHS):
    resnet.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    resnet.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = resnet(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(resnet.state_dict(), "best_resnet50.pth")

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

print(f"✅ Best Validation Accuracy: {best_val_acc:.2f}%")

Epoch 1/15, Loss: 2.9993, Val Acc: 24.74%
Epoch 2/15, Loss: 2.1187, Val Acc: 41.67%
Epoch 3/15, Loss: 1.7088, Val Acc: 43.49%
Epoch 4/15, Loss: 1.5390, Val Acc: 43.75%
Epoch 5/15, Loss: 1.3724, Val Acc: 46.35%
Epoch 6/15, Loss: 1.2841, Val Acc: 47.92%
Epoch 7/15, Loss: 1.1644, Val Acc: 47.14%
Epoch 8/15, Loss: 1.0716, Val Acc: 50.00%
Epoch 9/15, Loss: 0.9926, Val Acc: 48.18%
Epoch 10/15, Loss: 0.9739, Val Acc: 50.78%
Epoch 11/15, Loss: 0.9346, Val Acc: 48.96%
Epoch 12/15, Loss: 0.8630, Val Acc: 49.48%
Epoch 13/15, Loss: 0.8247, Val Acc: 46.09%
Epoch 14/15, Loss: 0.7997, Val Acc: 53.12%
Epoch 15/15, Loss: 0.7959, Val Acc: 47.40%
✅ Best Validation Accuracy: 53.12%


## Test Evaluation

In [7]:
resnet.load_state_dict(torch.load("best_resnet50.pth"))
resnet.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = resnet(imgs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"🎯 Final Test Accuracy: {100 * correct / total:.2f}%")

/tmp/ipykernel_9134/1161892318.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("best_resnet50.pth"))


🎯 Final Test Accuracy: 53.77%
